In [ ]:
import pprint
from json import loads
from IPython.display import display

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
numbers = sc.parallelize(range(0, 1000000))
rdd = sc.textFile('beatles/artists.json')
rdd = rdd.map(loads)

In [ ]:
numbers.count()

In [ ]:
rdd.count()

In [ ]:
rdd.collect()

In [ ]:
rdd.map(lambda l: l['track_name']).distinct().count()

In [ ]:
def tokenize(lyric):
    return lyric['track_name'].split()

beatles = rdd.flatMap(tokenize)

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.map(lambda word: (word, 1))

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.reduceByKey(lambda a, b: a + b)

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.map(lambda x:(x[1],x[0]))

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.sortByKey(False)

In [ ]:
beatles.collect()

In [ ]:
rdd.filter(lambda lyric: 'love' in lyric['track_name'].lower()).count()

In [ ]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)

In [ ]:
df_beatles = sql.createDataFrame(rdd)

In [ ]:
df_json = sql.read.json('beatles/artists.json')

In [ ]:
df_beatles.printSchema()

In [ ]:
df_beatles.columns

In [ ]:
df_beatles.toPandas()

In [ ]:
df_beatles.filter("track_duration >  300000").toPandas()

In [ ]:
df_beatles.filter(df_beatles.track_duration >  250000).count()

In [ ]:
df_beatles.registerTempTable('beatles')

In [ ]:
sql.sql('select count(*) as count from beatles').show()

In [ ]:

sql.sql('select album_name, count(track_name) as count from beatles group by album_name order by count').show(n=100)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_beatles.select(df_beatles.album_name) \
    .agg(F.count(df_beatles.track_name)) \
    .collect()